# Data Preparation Exercise 1

-----------------------
* 目標：讀取並且清理2017年環保署所有空氣品質觀測站的資料
    * 我們以 2015 年**花東空品區**的資料做示範
    * 請依據資料回答作業中的問題

* 資料來源：[環保署空品監測網](https://taqm.epa.gov.tw/taqm/tw/)觀測站資料
  * [下載網址](https://taqm.epa.gov.tw/taqm/tw/YearlyDataDownload.aspx)

<img class=center src=figures/epa.jpg height= 450/>

## 資料下載

- 在歷年資料的[下載網址](https://taqm.epa.gov.tw/taqm/tw/YearlyDataDownload.aspx)可以找到 1982 至今的觀測資料

<img class=center src=figures/epa_download_page.jpg height= 450/>


## 原始資料內容

- 下載的檔案為壓縮檔，每個測站的資料有兩個格式，外加兩個格式的說明檔

<img class=center src=figures/epa_zipped_files.jpg height= 450/>

## 讀取資料

pandas.read_excel() 可以直接讀取 Excel 檔案，轉換成 `pandas.DataFrame`。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('../data/104年花蓮站_20160320.xls')
data.head()

In [ ]:
data.shape

## 修改資料

環保署資料的欄位名稱是中文跟數字，為了方便後續處理，我們可以先修改欄位名稱

In [ ]:
data.columns = ['date','station','item','h00','h01','h02','h03','h04','h05','h06','h07','h08','h09',
                'h10','h11','h12','h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']
data.head(10)

## 檢查資料讀取是否正確

從前10行資料看來，觀測的品項相當多，我們可以檢查一下與資料說明文件裡提到的21個項目是否相符：

|測項簡稱   |單位  |測項名稱|
|----------|-----|-------|
|SO2	|ppb	|二氧化硫|
|CO	|ppm	|一氧化碳|
|O3	|ppb	|臭氧|
|PM10	|μg/m3	|懸浮微粒|
|PM2.5	|μg/m3	|細懸浮微粒|
|NOX	|ppb	|氮氧化物|
|NO	|ppb	|一氧化氮|
|NO2	|ppb	|二氧化氮|
|THC	|ppm	|總碳氫合物|
|NMHC	|ppm	|非甲烷碳氫化合物|
|CH4	|ppm	|甲烷|
|UVB	|UVI	|紫外線指數|
|AMB_TEMP	|℃	|大氣溫度|
|RAINFALL	|mm	|雨量|
|RH	|%	|相對溼度|
|WIND_SPEED	|m/sec	|風速(以每小時最後10分鐘算術平均)|
|WIND_DIREC	|degress	|風向(以每小時最後10分鐘向量平均)|
|WS_HR	|m/sec	|風速小時值(以整個小時算術平均)|
|WD_HR	|degress	|風向小時值(以整個小時向量平均)|
|PH_RAIN	|pH	|酸鹼值(酸雨)|
|RAIN_COND	|μS/cm	|導電度(酸雨)|

In [ ]:
pd.crosstab(data['item'], 'count')

從結果來看，花蓮測站的資料裡只有17個觀測品項，但是每個都有365天的數據。

## 檢查資料讀取是否正確

從說明文件和前幾列資料內容來觀察，這筆資料前三個欄位應該是文字，後面24個欄位應該是數字，讓我們檢查一下檔案讀取的是否正確

In [ ]:
data.dtypes

## 發現問題

pandas.DataFrame 的資料型態， object 通常是 str 或是更複雜的物件，而不是數字，顯然資料讀取過程有出現問題。

然而，問題發生的原因是什麼？為什麼系統會自動把數值的欄位辨識成字元呢？

- 打開原始資料逐筆檢查當然是一種方法，但是用人工檢查 6205x24 個數字，似乎不是很實際
- 我們可以嘗試把應該是數字的資料轉換成 flaot，看看會出現什麼錯誤訊息

`floatdata = data.iloc[:,3:]
floatdata.astype(np.float32)`

<img class=center src=figures/error_casting.jpg height= 450/>

## 發現問題

從上面的嘗試，我們看到的錯誤訊息是：`ValueError: could not convert string to float: 'NR'`。可見我們理應是數值資料的欄位裡出現了 NR 這個字串。

如果查閱資料說明，我們在「10.普通測站資料註記說明」會看到：

|符號|說明|
|---|------------------|
|\# |表示儀器檢核為無效值 |
|* |表示程式檢核為無效值 |
|x |表示人工檢核為無效值|
|NR |表示無降雨|
|空白 |表示缺值|

所以，除了空白資料 pandas 會自動辨識為遺失資料之外，其他的符號，我們需要人工指定為遺失值。

## 尋找 pandas.DataFrame 裡的字串

Python 支援 regular expression，可以協助我們偵測資料裡的字串是否包含特定的字元。

In [ ]:
import re

a = ['1','2','3#','4','5*','6','7x','8','NR','10']

def check_special_str(x):
    return(re.findall('\#|\*|x|NR', x) != [])

[check_special_str(x) for x in a]

In [ ]:
def check_special_str2(x):
    if re.findall('\#|\*|x|NR', x) != []:
        return(np.nan)
    else:
        return(x)
[check_special_str2(x) for x in a]

## 處理遺失值

我們可以把前面例子的作法，套用在我們的資料上。讓我們先定義兩個函數，分別偵測「遺失值」跟「無降水」，把數值分別設為 `np.nan` 跟 `0`。

In [ ]:
def detect_epa_nan(x):
    if re.findall('\#|\*|x', str(x))!=[]:
        return(np.nan)
    else:
        return(x)

def detect_epa_norain(x):
    if str(x)=='NR':
        return(0)
    else:
        return(x)

floatdata = floatdata.applymap(detect_epa_nan)
floatdata = floatdata.applymap(detect_epa_norain)
floatdata.astype(np.float32)
floatdata.head()

## 將處理過的資料與原始資料合併

假設我們並不關心24小時內的變化，只關心每日的平均值，我們可以用處理過的資料計算每個觀測品項的日平均值：

In [ ]:
data.iloc[:,3:] = floatdata
daily_data = data.loc[:,['date','station','item']]
daily_data['daily_mean'] = floatdata.mean(axis=1)
daily_data.head()

## 選取特定資料

我們也可以挑選特定日期的特定觀測品項，觀察其24小時的變化。

In [ ]:
data.loc[(data['date']=='2015/07/31') & (data['item']=='PM2.5'),:]

In [ ]:
a = data.loc[(data['date']=='2015/07/31') & (data['item']=='PM2.5'),:]
a = np.array(a.iloc[:,3:]).flatten()
plt.plot(a)

## 選取特定資料

我們也可以從日平均資料裡挑選特定觀測品項，觀察其一年的的變化。

In [ ]:
pm25 = daily_data.loc[daily_data['item']=='PM2.5',['date','daily_mean']]
pm25.reset_index(inplace=True)
plt.plot(pm25['daily_mean'])

## 格式重整

原始資料提供的格式，是以「測站─日期─測項」為索引（`key`），24個小時為欄位，但是在某些情況下，我們需要的是以「測站─日期─小時」為索引，22個測項為欄位，這時候就需要資料重整。

In [ ]:
tmp = data.loc[data['item']=='O3',:]
o3 = pd.melt(tmp, id_vars=['date','station'], value_vars=tmp.keys()[3:], var_name='hour', value_name='O3')
o3 = o3.sort_values(['date', 'hour'])
o3.head()

透過類似的格式重整，我們可以把一個測站的多個測項變成獨立的欄位，也可以把多個測站的相同測項合併成單一的資料表。

## 用函數加速格式重整

假設我們要把花蓮測站的所有測項轉換成如前例中的獨立時間序列，然後合併成一個表格，我們可以用函數簡化這個過程：

In [ ]:
# Retrieve one item from EPA data and form a time series
def retrieve_epa_item(data, var):
    tmp = data.loc[data['item']==var,:]
    ts = pd.melt(tmp, id_vars=['date'], value_vars=tmp.keys()[3:], var_name='hour', value_name=var)
    return(ts)

co = retrieve_epa_item(data, 'CO')
co.head()

## 用函數加速格式重整 (2)

然後我們可以用迴圈對每個測項執行上面的函數，然後合併資料集：

In [ ]:
# 所有測項
items = list(set(data['item']))

# Create the 1st dataframe
newdata = retrieve_epa_item(data, items[0])

# Loop through the rest of items
for i in items[1:]:
    tmp = retrieve_epa_item(data, i)
    newdata = newdata.merge(tmp, on=['date','hour'], how='left')

# Sort with date-hour to make the time-series in order
newdata = newdata.sort_values(['date', 'hour'])
newdata.head()

## 作業練習

- 請讀取104年花東空品區三個測站的資料，進行分析，並回答[以下問題](https://docs.google.com/forms/d/e/1FAIpQLSeBFJKvKnAX6ajU-sYRFEkVBBfaU9NpFvTbinDtb29opZ1dHg/viewform?usp=sf_link)
- 請讀取104年全部觀測站的資料，進行分析，並回答最後兩題
- 作業連結：https://goo.gl/forms/ARwizVHTLZqWkAH03
- 資料下載：http://140.112.67.93/downloads/
  - [104年花東空品區三個測站的資料：104_HOUR_07_20160323.zip](http://140.112.67.93/downloads/104_HOUR_07_20160323.zip)
  - [104年全部觀測站的資料：104_HOUR_00_20160323.zip](http://140.112.67.93/downloads/104_HOUR_00_20160323.zip)
- 請在 2018/12/01 00:00 (UTC+8) 之前完成作答，逾時不計分。